In [1]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json

import time

# data gathering
* read from `twitter-archive-enhanced.csv`
* download `image_predictions.tsv`

* use tweepy to load
    - retweet count
    - like count
    - ...
* save tweater data to `tweet_json.txt` and load them as df

#### read from `twitter-archive-enhanced.csv`

In [2]:
tweater_acrchive = pd.read_csv('twitter-archive-enhanced.csv')

#### download `image_predictions.tsv`

In [3]:
response = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

In [4]:
with open('image_predictions.tsv','wb') as file:
    file.write(response.content)

image_predictions = pd.read_csv('image_predictions.tsv',sep='\t')

* use tweepy to load
    - retweet count
    - like count
    - ...


In [6]:
# consumer_key = 'YOUR CONSUMER KEY'
# consumer_secret = 'YOUR CONSUMER SECRET'
# access_token = 'YOUR ACCESS TOKEN'
# access_secret = 'YOUR ACCESS SECRET'

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)

## http://docs.tweepy.org/en/v3.2.0/api.html#API

# api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

- https://tweepy.readthedocs.io/en/latest/api.html#API.get_status
- http://docs.tweepy.org/en/latest/extended_tweets.html?highlight=tweet_mode#extended-mode
- http://docs.tweepy.org/en/latest/extended_tweets.html?highlight=tweet_mode#examples

In [7]:
# for tweet_id in tweater_acrchive.tweet_id:
#     tweet = api.get_status(tweet_id, tweet_mode='extended')
#     print(tweet.text)
#     break

* save tweater data to `tweet-json.txt` 
* convert data to dict then to df

In [8]:
## https://stackabuse.com/reading-and-writing-json-to-a-file-in-python/
# with open(tweet-json.txt,'wb') as file:
#     json.dump()

In [7]:
## temporary
rows = []
with open('tweet-json.txt','r') as json_file:
    for line in json_file.readlines():
        data = json.loads(line)
        data['favorite_count']
        
        rows.append({'tweet_id': data['id'],
                     'retweet_count':  data['retweet_count'],
                     'favorite_count': data['favorite_count'],
                     'hashtags': data['entities']['hashtags']})

In [8]:
tweeter_extra = pd.DataFrame(rows)

### test loaded data

In [10]:
tweater_acrchive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [12]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [13]:
tweeter_extra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2354 non-null   int64 
 1   retweet_count   2354 non-null   int64 
 2   favorite_count  2354 non-null   int64 
 3   hashtags        2354 non-null   object
dtypes: int64(3), object(1)
memory usage: 73.7+ KB


# Assessing Data

### check any error in rating , dog stages

```
I extracted this data programmatically, but I didn't do a very good job. The ratings probably aren't all correct. Same goes for the dog names and probably dog stages (see below for more information on these) too.
```




In [90]:
st =  tweater_acrchive.rating_numerator.astype(str) + '/' + tweater_acrchive.rating_denominator.astype(str)

In [95]:
to = tweater_acrchive.text.str.extract(r'(\d+\/\d+)')[0]

In [96]:
tweater_acrchive[st!=to].values

array([[835246439529840640, 8.35245984028504e+17, 26259576.0,
        '2017-02-24 21:54:03 +0000',
        '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
        "@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",
        nan, nan, nan, nan, 960, 0, 'None', 'None', 'None', 'None',
        'None'],
       [826598799820865537, 8.265983652700078e+17, 4196983835.0,
        '2017-02-01 01:11:25 +0000',
        '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
        "I was going to do 007/10, but the joke wasn't worth the &lt;10 rating",
        nan, nan, nan, nan, 7, 10, 'None', 'None', 'None', 'None',
        'None']], dtype=object)

* `835246439529840640` ratings at end of text

* `826598799820865537` is was comment , was rated in other post

https://twitter.com/dog_rates/status/826598799820865537

In [100]:
tweater_acrchive[['rating_numerator','rating_denominator']].describe()

,rating_numerator,rating_denominator
count,2356.000000,2356.000000
mean,13.126486,10.455433
std,45.876648,6.745237
min,0.000000,0.000000
25%,10.000000,10.000000
50%,11.000000,10.000000
75%,12.000000,10.000000
max,1776.000000,170.000000


In [122]:
small_rate = tweater_acrchive[tweater_acrchive.rating_numerator < 10]

In [123]:
small_rate[['text','rating_numerator','rating_denominator']].values

array([['This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948',
        5, 10],
       ["This is Jerry. He's doing a distinguished tongue slip. Slightly patronizing tbh. You think you're better than us, Jerry? 6/10 hold me back https://t.co/DkOBbwulw1",
        6, 10],
       ["When you're so blinded by your systematic plagiarism that you forget what day it is. 0/10 https://t.co/YbEJPkg4Ag",
        0, 10],
       ...,
       ['Here is a very happy pup. Big fan of well-maintained decks. Just look at that tongue. 9/10 would cuddle af https://t.co/y671yMhoiR',
        9, 10],
       ['This is a western brown Mitsubishi terrier. Upset about leaf. Actually 2 dogs here. 7/10 would walk the shit out of https://t.co/r7mOb2m0UI',
        7, 10],
       ['Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj',
        8, 10]], dtype=ob

* rating with fraction has mistakes

In [126]:
small_dem = tweater_acrchive[tweater_acrchive.rating_denominator < 10]
small_dem[['tweet_id','text','rating_numerator','rating_denominator']].values

array([[835246439529840640,
        "@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",
        960, 0],
       [810984652412424192,
        'Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx',
        24, 7],
       [666287406224695296,
        'This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv',
        1, 2]], dtype=object)

* 810984652412424192 not rating 
* 666287406224695296 rating at the end

In [132]:
##  already reported
big_dem = tweater_acrchive[tweater_acrchive.rating_numerator > 20]
big_dem[['tweet_id','text','rating_numerator','rating_denominator']].values

array([[855862651834028034,
        '@dhmontgomery We also gave snoop dogg a 420/10 but I think that predated your research',
        420, 10],
       [855860136149123072,
        '@s8n You tried very hard to portray this good boy as not so good, but you have ultimately failed. His goodness shines through. 666/10',
        666, 10],
       [838150277551247360, '@markhoppus 182/10', 182, 10],
       [835246439529840640,
        "@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",
        960, 0],
       [832215909146226688,
        "RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…",
        75, 10],
       [820690176645140481,
        'The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd',
        84, 70],
       [810984652412424192,
        'Meet Sam. She smiles 24/7 &amp; secretly aspires to be a rei

### messy
- tweet extra
- wrong dtypes
    * tweater_archive: timestamp,retweeted_status_timestamp
- dog type in coulmn name (doggo  floofer pupper puppo)

### dirty
* tweater_archive
    - compex values in source coloumn 
    - tweat `[826598799820865537 , 810984652412424192 ]` is not rating tweete
    - rating with fraction has mistakes _ex.. `9.5/10` is saved as `5/10`_
    - tweets with rating at the end
    ex..` [835246439529840640, 666287406224695296] `

In [16]:
tweater_acrchive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 